# clase ST7017 NLP Aplicado
# diferentes ejercicios de word embeddings para la lectura 03


In [ ]:
# representación clásica en BoW
from sklearn.feature_extraction.text import CountVectorizer
corpus = ["el gato está en la casa", "el perro está en el jardín"]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray())
print(vectorizer.get_feature_names_out())


In [ ]:
# # representación clásica en word2vec

from gensim.models import Word2Vec

sentences = [
    ["el", "gato", "está", "en", "la", "casa"],
    ["el", "perro", "está", "en", "el", "jardín"]
]
model = Word2Vec(sentences, vector_size=50, window=2, min_count=1, sg=1)

print(model.wv["gato"])
print(model.wv.similarity("gato", "perro"))


In [ ]:
# carga de modelos preentrenados
# GloVe

from gensim.downloader import load
model = load("glove-wiki-gigaword-50")  # vectores de 50 dimensiones
print(model["king"])
print(model.most_similar("king"))


In [ ]:
print(model["python"])
print(model.most_similar("python"))

In [ ]:
# visualización en 2D 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

words = ["king", "queen", "man", "women"]
vectors = [model[w] for w in words]

pca = PCA(n_components=2)
coords = pca.fit_transform(vectors)

for word, (x, y) in zip(words, coords):
    plt.scatter(x, y)
    plt.text(x+0.01, y+0.01, word)
plt.show()


In [ ]:
# ejercicio de clase:
'''
Ejemplo1:
Dado un conjunto de palabras ‘similares’ en contexto, detectar las palabras fuera del contexto:
palabras = ["pera", "piña", "papaya", "carro"]

respuesta: "carro"

palabras = ["peach", "pineapple", "papaya", "car"]

respuesta: "car"

'''


In [ ]:
# cargar modelo preentrenado glove

import gensim.downloader as api

# escoja un modelo:

# modelo Glove
#model = api.load("glove-wiki-gigaword-50")

# modelo word2vec

# model = api.load("word2vec-google-news-300")



In [ ]:
# palabras:

# prueba con palabras en español y con palabras en inglés y analice los resultados

# palabras en español
#palabras = ["pera", "piña", "papaya", "carro"]

# palabras en ingles
palabras = ["peach", "pineapple", "papaya", "car"]

In [ ]:
# verificar las palabras en el modelo:

palabras_validas = [w for w in palabras if w in model]
print(palabras_validas)


In [ ]:
# calcular distancias al centroide de los 4 embeddings

import numpy as np
vectors = [model[w] for w in palabras_validas]
print(vectors)
centroid = np.mean(vectors, axis=0)

distancias = [np.linalg.norm(model[w] - centroid) for w in palabras_validas]
outlier = palabras_validas[np.argmax(distancias)]
print("Palabra fuera de contexto:", outlier)


# clustering con representación de embeddings y modelos clásicos no supervisados de ML

In [ ]:
# descargar y descomprimir modelo fasttext con corpus en español
# 1.3 GB
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz

# 4.5 GB
!gunzip cc.es.300.vec.gz

In [ ]:
# cargar el modelo desde el archivo descargado previamente
import gensim

# Cargar modelo FastText para español (debe estar descargado)
model = gensim.models.KeyedVectors.load_word2vec_format("cc.es.300.vec", binary=False)


In [ ]:
palabras = [
    # objetos de casa
    "mueble", "nevera", "lavadora", "silla", "sofa", "tv",
    # colores
    "rojo", "amarillo", "verde", "azul", "gris", "negro",
    # Vehículos
    "auto", "camión", "moto", "bicicleta", "avión", "barco"
]



In [ ]:
# verificar que las palabras si estén en el corpus del modelo, filtrar las que no están
X = [model[word] for word in palabras if word in model]
palabras_filtradas = [word for word in palabras if word in model]
print(palabras_filtradas)

In [ ]:
# ejecutar un modelo clásico de kmeans con 3 grupos
from sklearn.cluster import KMeans

k = 3  # número esperado de grupos
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(X)


In [ ]:
# visualizar en 2D grupos
# se utiliza PCA para poder proyectar en un espacio 2D vectores densos y conservar las relaciones de distancia
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reducción de dimensión
pca = PCA(n_components=2)
coords = pca.fit_transform(X)

# Visualización
plt.figure(figsize=(10, 7))
for i, word in enumerate(palabras_filtradas):
    plt.scatter(coords[i, 0], coords[i, 1], c=f"C{labels[i]}")
    plt.text(coords[i, 0] + 0.01, coords[i, 1] + 0.01, word)
plt.title("Visualización de distancias WE+PCA+K-Means)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.grid(True)
plt.show()


In [ ]:
# analice los resultados y plantee otro dataset de palabras que SI permita tener 3 grupos bien identificados
